# ETL output_steam_games.json 

## Importando librerías 

In [39]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq


## Extracción de data y Exploración 

In [4]:
#extraccion del json
row = [] #lista vacia para ir guardando las filas
with open ("data/output_steam_games.json") as file: #utilizo with para que el archivo se abra y cierre
    for line in file.readlines(): #bucle para ir leyendo filas y luego agregarlas a row
        data = json.loads(line)
        row.append(data)

# genero el dataframe 
games = pd.DataFrame(row)
games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561197970982479,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,js41637,76561198035864385,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evcentric,76561198007712555,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Riot-Punch,76561197963445855,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,doctr,76561198002099482,"[{'item_id': '300', 'item_name': 'Day of Defea...",541.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,False,773640,NaN,"Nikita ""Ghost_RUS""",NaN,NaN,NaN,NaN
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,NaN,Sacada,NaN,NaN,NaN,NaN
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich,NaN,NaN,NaN,NaN
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,NaN,"xropi,stev3ns",NaN,NaN,NaN,NaN


In [5]:
games.shape

(120445, 19)

In [6]:
games.describe()

,discount_price,items_count
count,225.000000,88310.000000
mean,11.930533,58.353629
std,17.492643,122.312095
min,0.490000,0.000000
25%,1.390000,3.000000
50%,4.190000,26.000000
75%,22.660000,73.000000
max,139.990000,7762.000000


In [7]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  object 
 12  id              32133 non-null  object 
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
 15  user_id         88310 non-null  object 
 16  steam_id        88310 non-null  object 
 17  items           88310 non-nul

### Existen columnas que no se utilizan por lo tanto se eliminan con el fin de optimizar el tratamiento de los datos

In [8]:
#elimino las columnas que no estan en el diccionario de datos
steam_games = games.drop(columns = ["user_id", "steam_id","items", "items_count"])
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,False,773640,NaN,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,NaN,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,NaN,"xropi,stev3ns"


### Busco nulos y duplicados

In [9]:
nulos_por_columna=steam_games.isnull().sum()
nulos_por_columna


publisher          96362
genres             91593
app_name           88312
title              90360
url                88310
release_date       90377
tags               88473
reviews_url        88312
discount_price    120220
specs              88980
price              89687
early_access       88310
id                 88312
metascore         117768
developer          91609
dtype: int64

In [10]:
Total_nulos= steam_games.isnull().sum().sum()
Total_nulos

1406985

In [11]:
#el dataframe contiene el 80% de datos nulos por lo que se eliminan
steam_games = steam_games.dropna(how="all").reset_index(drop=True)
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,NaN,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,NaN,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,0.83,[Single-player],0.99,False,767400,NaN,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,1.79,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,False,773640,NaN,"Nikita ""Ghost_RUS"""
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,NaN,Sacada
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,NaN,"xropi,stev3ns"


In [12]:
#busco y analizo los duplicados
columnas_para_verificar = ['id']
duplicados_columnas = steam_games[steam_games.duplicated(subset=columnas_para_verificar)]
duplicados_columnas



,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,86,Machine Games
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,91,"Rocksteady Studios,Feral Interactive (Mac)"


In [13]:
steam_games[steam_games["developer"]== "Machine Games"]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
12237,Bethesda Softworks,[Action],Wolfenstein II: The Freedom Chronicles - Seaso...,Wolfenstein II: The Freedom Chronicles - Seaso...,http://store.steampowered.com/app/650410/Wolfe...,2017-11-07,"[Gore, Action, Violent]",http://steamcommunity.com/app/650410/reviews/?...,NaN,"[Single-player, Downloadable Content, Full con...",24.99,False,650410,NaN,Machine Games
13318,Bethesda Softworks,[Action],Wolfenstein II: The Freedom Chronicles - Episo...,Wolfenstein II: The Freedom Chronicles - Episo...,http://store.steampowered.com/app/624620/Wolfe...,2017-12-13,"[Action, Violent]",http://steamcommunity.com/app/624620/reviews/?...,NaN,"[Single-player, Downloadable Content, Steam Ac...",9.99,False,624620,NaN,Machine Games
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,86,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,86,Machine Games
28996,Bethesda Softworks,[Action],Wolfenstein: The New Order,Wolfenstein: The New Order,http://store.steampowered.com/app/201810/Wolfe...,2014-05-19,"[FPS, Action, Shooter, Alternate History, Firs...",http://steamcommunity.com/app/201810/reviews/?...,NaN,"[Single-player, Steam Achievements, Partial Co...",19.99,False,201810,81,Machine Games


#### La fila duplicada 14573 es idéntica a la 13894.. por lo que podemos eliminarla

In [14]:
steam_games[steam_games["developer"]== "Rocksteady Studios,Feral Interactive (Mac)"]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,200260,91,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,91,"Rocksteady Studios,Feral Interactive (Mac)"
31617,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,http://store.steampowered.com/app/35140/Batman...,2010-03-26,"[Action, Batman, Stealth, Adventure, Third Per...",http://steamcommunity.com/app/35140/reviews/?b...,NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,35140,91,"Rocksteady Studios,Feral Interactive (Mac)"


#### La fila duplicada 30961, es idéntica a la 1068; pero contiene el dato de id nulo, mientras que la fila 1068 posee el id completo 200260. Por lo que se elimina la fila duplicada 30961 

In [15]:
#elimino columnas con id duplicadas
duplicados_col = [14573, 30961]
steam_games= steam_games.drop(duplicados_col)

In [16]:
columnas_para_verificar = ['id']
duplicados_columnas = steam_games[steam_games.duplicated(subset=columnas_para_verificar)]
duplicados_columnas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer


In [17]:
steam_games.shape

(32133, 15)

## Transformación de los datos 

In [19]:
# conozco que tipos de datos tiene el dataset para saber cual debo modificar
tipos_de_datos=steam_games.dtypes
tipos_de_datos

publisher          object
genres             object
app_name           object
title              object
url                object
release_date       object
tags               object
reviews_url        object
discount_price    float64
specs              object
price              object
early_access       object
id                 object
metascore          object
developer          object
dtype: object

#### EL dataset aún es un archivo json, por lo que el tipo de dato object , lo que puede ser datos anidados con formato clave - valor. Entonces a través de un bucle se analiza columna por columna y se almacenan en un diccionario vacio para luego transformarlo en un dataset y alli poder ver el tipo de datos de las columnas.

In [20]:
tipo_data = {"columna":[],"tipos_de_datos":[]} #genero un diccionario vacio para ir almacenando lo que genere el bucle

for columna in steam_games.columns: #un bucle que va recorriendo 
    tipo_data["columna"].append(columna)
    tipo_data["tipos_de_datos"].append(steam_games[columna].apply(type).unique())

analisis= pd.DataFrame(tipo_data)
analisis

,columna,tipos_de_datos
0,publisher,"[<class 'str'>, <class 'float'>]"
1,genres,"[<class 'list'>, <class 'float'>]"
2,app_name,"[<class 'str'>, <class 'float'>]"
3,title,"[<class 'str'>, <class 'float'>]"
4,url,[<class 'str'>]
5,release_date,"[<class 'str'>, <class 'float'>]"
6,tags,"[<class 'list'>, <class 'float'>]"
7,reviews_url,"[<class 'str'>, <class 'float'>]"
8,discount_price,[<class 'float'>]
9,specs,"[<class 'list'>, <class 'float'>]"


### Transformación de la columna "release_date"

#### Es necesario que pueda extraerse el año del dato fecha ; por lo que se transforma la columna "release_date", generando una con el año de estreno

In [21]:
#composición de la columna
steam_games["release_date"]

0        2018-01-04
1        2018-01-04
2        2017-07-24
3        2017-12-07
4               NaN
            ...    
32130    2018-01-04
32131    2018-01-04
32132    2018-01-04
32133    2017-09-02
32134           NaN
Name: release_date, Length: 32133, dtype: object

In [22]:
steam_games["release_date"].value_counts()

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: count, Length: 3582, dtype: int64

In [23]:
# Extraer el año de la columna "release_date" y reemplazar los valores nulos con "sin dato"
steam_games['Año_estreno'] = steam_games['release_date'].str.extract(r'(\d{4})').fillna('sin dato')

steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,Año_estreno
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,NaN,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,NaN,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,0.83,[Single-player],0.99,False,767400,NaN,彼岸领域,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,1.79,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,sin dato
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,False,773640,NaN,"Nikita ""Ghost_RUS""",2018
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,NaN,Sacada,2018
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich,2018
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,NaN,"xropi,stev3ns",2017


### Transformación de la columna "price"

#### La columna "price", posee valores nulos que deben ser reemplazados por valores numéricos de tipo flotante. Toda la columna debe ser dato de tipo flotante.

In [24]:
#observo la composición de la columna
steam_games["price"]

0                4.99
1        Free To Play
2        Free to Play
3                0.99
4                2.99
             ...     
32130            1.99
32131            4.99
32132            1.99
32133            4.99
32134            4.99
Name: price, Length: 32133, dtype: object

In [25]:
steam_games["price"].value_counts()

price
4.99           4278
9.99           3902
2.99           3429
0.99           2607
1.99           2541
               ... 
74.76             1
32.99             1
14.95             1
26.99             1
Free to Use       1
Name: count, Length: 162, dtype: int64

In [26]:
# se define una función para poder reemplazar los datos string de la columna "price" por flotantes 0.0

def cambio_a_float(valor):

    if pd.isna(valor): #esta parte de la función es para el caso de que no exista ningun string o float en el campo
        return 0.0
    try:
        flotante = float(valor) #si el valor es float lo conservo
        return flotante
    except (ValueError, TypeError): #si el valor es diferente retorno 0.0
        return 0.0
    
#Aplico esta función a la columna precio
steam_games["price"] = steam_games["price"].apply(cambio_a_float)


In [27]:
#corroboro el cambio de tipo de dato
steam_games["price"]

0        4.99
1        0.00
2        0.00
3        0.99
4        2.99
         ... 
32130    1.99
32131    4.99
32132    1.99
32133    4.99
32134    4.99
Name: price, Length: 32133, dtype: float64

In [28]:
#buscamos outliers en precio... aunque se puede comprender que los valores resultarán muy diferentes, debido a ofertas o juegos gratis.

# Se calcula el IQR (Rango Intercuartil)
Q1 = steam_games['price'].quantile(0.25)
Q3 = steam_games['price'].quantile(0.75)
IQR = Q3 - Q1
print(Q1)
print(Q3)
print(IQR)

1.99
9.99
8.0


In [29]:
#Se defiene el umbral de outliers
umbral_superior = Q3 + 1.5 * IQR
umbral_inferior = Q1 - 1.5 * IQR
print(umbral_superior)
print(umbral_inferior)

21.990000000000002
-10.01


In [30]:
#Se identifican los outliers
outliers_superiores = steam_games[steam_games['price'] > umbral_superior]
outliers_inferiores = steam_games[steam_games['price'] < umbral_inferior]
print(outliers_superiores)
print(outliers_inferiores)

                              publisher   
8                Poppermost Productions  \
18               Poppermost Productions   
51                                  NaN   
59                               SimBin   
234                              NCsoft   
...                                 ...   
31938  Firedog Creative Company Limited   
31956                               NaN   
31959                 CarX Technologies   
32070                  DreamInCode B.V.   
32098                        Koch Media   

                                                  genres   
8              [Free to Play, Indie, Simulation, Sports]  \
18             [Free to Play, Indie, Simulation, Sports]   
51                                                   NaN   
59                                              [Racing]   
234                         [Massively Multiplayer, RPG]   
...                                                  ...   
31938                                  [RPG, Simulation]   
319

In [31]:
outliers_inferiores

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,Año_estreno


In [32]:

outliers_superiores

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,Año_estreno
8,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Legend Pass,SNOW - All Access Legend Pass,http://store.steampowered.com/app/774278/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774278/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",29.99,False,774278,NaN,Poppermost Productions,2018
18,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - Lifetime Pack,SNOW - Lifetime Pack,http://store.steampowered.com/app/774279/SNOW_...,2018-01-03,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774279/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",59.99,False,774279,NaN,Poppermost Productions,2018
51,NaN,NaN,Medicalholodeck Personal Version,NaN,http://store.steampowered.com/app/769350/Medic...,NaN,[Education],http://steamcommunity.com/app/769350/reviews/?...,NaN,"[Tracked Motion Controllers, Seated, Standing]",29.99,False,769350,NaN,NaN,sin dato
59,SimBin,[Racing],RACE - The WTCC Game,RACE - The WTCC Game,http://store.steampowered.com/app/4230/RACE__T...,2006-11-24,"[Racing, Simulation]",http://steamcommunity.com/app/4230/reviews/?br...,NaN,"[Single-player, Multi-player]",39.99,False,4230,NA,SimBin,2006
234,NCsoft,"[Massively Multiplayer, RPG]",Guild Wars Trilogy,Guild Wars Trilogy,http://store.steampowered.com/app/29570/Guild_...,2008-09-25,"[RPG, Massively Multiplayer, MMORPG, PvP]",http://steamcommunity.com/app/29570/reviews/?b...,NaN,"[Multi-player, MMO]",29.99,False,29570,NaN,ArenaNet®,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31938,Firedog Creative Company Limited,"[RPG, Simulation]",愛神餐館2,愛神餐館2,http://store.steampowered.com/app/769680/2/,2017-12-29,"[RPG, Simulation, Anime]",http://steamcommunity.com/app/769680/reviews/?...,31.49,[Single-player],34.99,False,769680,NaN,Firedog Creative Company Limited,2017
31956,NaN,NaN,Three Kingdoms VR - Jade Knight (光之三國VR - 青龍騎),NaN,http://store.steampowered.com/app/731120/Three...,NaN,"[Action, Casual, Simulation, Sports]",http://steamcommunity.com/app/731120/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Clou...",24.99,False,731120,NaN,NaN,sin dato
31959,CarX Technologies,"[Massively Multiplayer, Racing, Simulation, Sp...",CarX Drift Racing Online - Deluxe,CarX Drift Racing Online - Deluxe,http://store.steampowered.com/app/776950/CarX_...,2017-12-28,"[Massively Multiplayer, Simulation, Racing, Sp...",http://steamcommunity.com/app/776950/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",29.99,False,776950,NaN,CarX Technologies,2017
32070,DreamInCode B.V.,"[Action, Adventure, Casual, Indie, Simulation,...",Mesozoica,Mesozoica,http://store.steampowered.com/app/719590/Mesoz...,2018-01-02,"[Early Access, Simulation, Strategy, Casual, I...",http://steamcommunity.com/app/719590/reviews/?...,25.49,[Single-player],29.99,True,719590,NaN,"DreamInCode B.V.,Squadron Interactive",2018


### Transformación de la columna "genres"

#### Terminada la transformación de los datos numéricos, se trata la columna "genres"

In [33]:
steam_games["genres"][0]

['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']

In [34]:
#con explode hago que la columna genres tome cada valor que tiene almacenado y lo devuelva en un campo nuevo pero conservando el mismo id
steam_games = steam_games.explode("genres")
steam_games = steam_games.dropna(subset=["genres"])
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,Año_estreno
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
0,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
0,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
0,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
0,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32132,Laush Studio,Indie,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich,2018
32132,Laush Studio,Racing,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich,2018
32132,Laush Studio,Simulation,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich,2018
32133,SIXNAILS,Casual,EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,NaN,"xropi,stev3ns",2017


#### Procedo a completar los datos que se encuentran nulos o vacios, para el caso de los datos de tipo string del dataset 

In [35]:
#busco espacios vacios o sin datos 
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71551 entries, 0 to 32133
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       59112 non-null  object 
 1   genres          71551 non-null  object 
 2   app_name        71549 non-null  object 
 3   title           71549 non-null  object 
 4   url             71551 non-null  object 
 5   release_date    71498 non-null  object 
 6   tags            71505 non-null  object 
 7   reviews_url     71551 non-null  object 
 8   discount_price  580 non-null    float64
 9   specs           70994 non-null  object 
 10  price           71551 non-null  float64
 11  early_access    71551 non-null  object 
 12  id              71551 non-null  object 
 13  metascore       5467 non-null   object 
 14  developer       71201 non-null  object 
 15  Año_estreno     71551 non-null  object 
dtypes: float64(2), object(14)
memory usage: 9.3+ MB


In [36]:
#inicio un archivo con las columnas que tengo que reemplazar nulos
archivo_columnas=["publisher", "app_name","title","developer"]

#genero un dataframe con las columnas y sus nulos reemplazados por "Sin Datos"
nulos= steam_games[archivo_columnas].fillna("Sin Datos")

#elimino las colummas a reemplazar y concateno ambos dataframes 
steam_games = pd.concat([steam_games.drop(archivo_columnas, axis=1), nulos], axis=1)
steam_games

,genres,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,Año_estreno,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32132,Indie,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,2018,Laush Studio,Russian Roads,Russian Roads,Laush Dmitriy Sergeevich
32132,Racing,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,2018,Laush Studio,Russian Roads,Russian Roads,Laush Dmitriy Sergeevich
32132,Simulation,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,2018,Laush Studio,Russian Roads,Russian Roads,Laush Dmitriy Sergeevich
32133,Casual,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,NaN,2017,SIXNAILS,EXIT 2 - Directions,EXIT 2 - Directions,"xropi,stev3ns"


In [37]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71551 entries, 0 to 32133
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   genres          71551 non-null  object 
 1   url             71551 non-null  object 
 2   release_date    71498 non-null  object 
 3   tags            71505 non-null  object 
 4   reviews_url     71551 non-null  object 
 5   discount_price  580 non-null    float64
 6   specs           70994 non-null  object 
 7   price           71551 non-null  float64
 8   early_access    71551 non-null  object 
 9   id              71551 non-null  object 
 10  metascore       5467 non-null   object 
 11  Año_estreno     71551 non-null  object 
 12  publisher       71551 non-null  object 
 13  app_name        71551 non-null  object 
 14  title           71551 non-null  object 
 15  developer       71551 non-null  object 
dtypes: float64(2), object(14)
memory usage: 9.3+ MB


## Guardo el dataset como csv

In [68]:
guardar = "data/steam_games.csv"

steam_games.to_csv(guardar, index=False, encoding="utf-8")

## Transformo csv a parquet para una optimización de los archivos

In [40]:
#Transformo el archivo csv a parquet
#Leo el archivo csv
steam_games= pd.read_csv("data/steam_games.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/steam_games.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(steam_games)
pq.write_table(table,output_file)